<a href="https://colab.research.google.com/github/sudotouchwoman/math-misc/blob/main/notebooks/simplex-method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Linear programming**

This sounds so weird at first and seems to be something trivial: we are used to solving linear problems, aren't we? Although most problems in our world tend to be nonlinear, the field of linear programming is widely explored and found applications in several domains, being the first major topic one studying operations research should understand.

However, this one was surprisingly complicated for me, concretely **the simplex method** (I prefer to call him flexing high-dimensional triangle). And I blame awful methodology and articles for not understanding the intuition behind this method.
Like, come on, this is linear algebra, we talk vectors and matrices here, why everyone (including my operations research teacher) starts with the tableu which cries out something like **"DAMN I AM SOMETHING OF A LINEAR SYSTEM MYSELF"**, but for unexpierenced reader this may take a while to get familiar with it

I am really thankful to the authors of these books:

+ [Bartazaa et. al., Linear programming and network  flows](https://www.academia.edu/23105646/Bazaraa_LP_430912_)

+ [Changhyun Kwon, Julia programming for operations research](https://www.softcover.io/read/7b8eb7d0/juliabook2/simplex) (the chapter on simplex method in particular)

I was inspired by this method and wanted to implement it. There was a great ground in the mentioned book for Julia, which is really akin to Python, so I used it as a reference. The first step was to understand the basic and non-basic variables along with basic feasible solutions.

Then there was the simplex table.

I see now! The key to the simplex method is that on each iteration we swap a single basic with the non-basic variable: this said, move to the neighbouring extreme point! (as the extreme point is defined by the basis, i.e. set of zeroed variables representing intersection of hyperplanes)

The point is (extreme), our cost function is linear and we can easily obtain its partial derivatives w.r. to any variable (it is just the corresponding factor in $c$ vector). Expressed algebraically, we iterativaly search for the minimal

$$
∂{z}/∂x_j=-(z_j-c_j)=\bar{c}<0
$$

The blocking variable is the leaving variable, as it becomes 0 in the end of pivoting (method iteration). If there is no blocking variable, the solution is unbound!


In [12]:
import numpy as np
from numpy.random import default_rng

In [2]:
class SimplexTableau:
  z_c : np.ndarray  # reduced costs vector
  Y : np.ndarray  # the main part of tableau
  x_B : np.ndarray  # basic variables values
  obj : np.number  # current objective function value
  b_idx : np.ndarray  # indices of basic variables

  def __init__(self, z_c, Y, x_B, b_idx, obj=.0) -> None:
      self.z_c = z_c
      self.Y = Y
      self.x_B = x_B
      self.obj = obj
      self.b_idx = b_idx


  def optimize(self, verbose=False) -> None:
    if verbose:
      self._print_tableau()
    while not self._is_optimal():
      self._pivoting()
      if not verbose: continue
      self._print_tableau()

    x_opt = np.zeros_like(self.z_c)
    x_opt[self.b_idx] = self.x_B
    if verbose:
      print(f'Optimal solution: ', *(f'{x:.3f} ' for x in x_opt))
      print(f'Objective value: {self.obj}')
    return x_opt, self.obj

  
  def _is_optimal(self) -> bool:
    return (self.z_c <= .0).all()


  def _pivoting(self) -> None:
    z_c = self.z_c
    Y = self.Y
    x_B = self.x_B
    obj = self.obj
    b_idx = self.b_idx

    # find the indices for
    # exiting and entering variables
    def pivot_point():

      # entering variable is the first positive
      # item in z_c; we look for its index
      # the index of entering variable
      entering = z_c.argmax()

      pos_idx = (Y[:, entering] > 0).squeeze()
      if not pos_idx.any(): raise RuntimeError('Unbounded problem')

      # min ratio test
      _ = (x_B[pos_idx] / Y[pos_idx, entering]).argmin()
      pos_idx = np.argwhere(pos_idx)  # convert bool-mask to indices
      exiting = pos_idx[_]

      return entering, exiting

    entering, exiting = pivot_point()

    # make pivot point unit
    coef = Y[exiting, entering]
    Y[exiting] /= coef
    x_B[exiting] /= coef

    # perform elementary operations
    # in order to make other `entering` entries
    # 0 except for exiting one in Y matrix
    for i, row in enumerate(Y):
      if i == exiting: continue
      coef = row[entering]
      row -= coef * Y[exiting].squeeze()
      # this is funny how python works
      # with numbers and references:
      # at first (in sake of beauty)
      # I tried for i, (row, x) in enumerate(zip(Y, x_B)):
      # such approach did not change the actual
      # values of x_B (as numbers are immutable, I guess)
      x_B[i] -= coef * x_B[exiting]

    # perform the same trick with z_c
    # (reduced costs vector)
    # and update the objective value
    coef = z_c[entering]
    z_c -= coef * Y[exiting].T.squeeze()
    obj -= coef * x_B[exiting]
    self.obj = obj[0]  # this one somehow broadcasts to a single-value array
    b_idx[exiting] = entering


  def _print_tableau(self) -> None:
    # pretty-print current state of the tableau
    print(f'Simplex Tableau')
    m, n = self.Y.shape
    # awkward formatting with generators, God I like these!
    hline = '+'.join(('-' * 9, '-' * 6 * n + '-', '-' * 9))

    print(hline)
    print(f"{' ' * 9}|", *(f'{z_c:5.2f}' for z_c in self.z_c), f"|{self.obj:5.2f}")
    print(hline)
    for b_idx, Y, x_B in zip(self.b_idx, self.Y, self.x_B):
      print(f'x[{b_idx:d}]{" " * 6}', *(f'{y:5.2f}' for y in Y), f"|{x_B:5.2f}")
    print(hline)


In [9]:
def simplex_method(A, b, c, task='min', verbose=False):
  # the method contains several steps:
  # 1) add missing slack variables
  # 2) create the tableau (this is just
  # a class representing used vectors in
  # a convenient way)
  # 3) perform the optimization:
  # this said, iteratively update and improve
  # the objective function value, i.e. seeked solution
  # return optimal solution and corresponding X vector
  # (note that this vector will include slack variables)

  if task not in ('min', 'max'):
    raise ValueError(
        'Solver performs either minimization or maximization. \
        Defaults to minimization')

  def add_slack_variables(A, b, c):
    # append identity matrix
    # to A and add corresponding
    # number of slots for slack variables
    # to c (these will be zeros initially)
    m, = b.shape

    I = np.eye(m)
    c_b = np.zeros(m)

    A = np.c_[A, I] # equivalent to A = np.hstack((A, I))
    c = np.r_[c, c_b] # equivalent to c = np.hstack((c, c_b))

    return A, b.T, c.T

  def make_tableau(A, b, c) -> SimplexTableau:
    A, b, c = map(np.asarray, (A, b, c))

    # if maximization problem given,
    # swap the components of c, i.e. x factors
    # note that one should inverse the sign
    # of the optimal function value in this case
    if task == 'max': c = -c

    m, p = A.shape
    n = m + p

    A, b, c = add_slack_variables(A, b, c)

    b_idx = np.arange(p, n)
    n_idx = np.arange(p)

    # extract the basis and the rest
    # actually, the basis is an identity matrix,
    # as inititally it only contains slack
    # variables
    B = A[:, b_idx]
    NB = A[:, n_idx]
    inv_B = np.linalg.inv(B)

    # this would be
    # equivalent to Y = I @ A = A
    # on the first step
    # but performed nevertheless is sake of
    # generality
    Y = inv_B @ A

    x_B = inv_B @ b
    c_B = c[b_idx]

    obj = c_B.T @ x_B

    z_c = np.zeros(n)
    z_c[n_idx] = c_B.T @ inv_B @ NB - c[n_idx]

    return SimplexTableau(z_c, Y, x_B, b_idx, obj)

  tableau = make_tableau(A, b, c)
  return tableau.optimize(verbose=verbose)

# **Examples**

Note that on each iteration there will be a basis of `m` variables, representing intersection of hyperplanes, or a BFS. 
The basis contains columns with single unit value and `0` elsewhere. 
Indices of basic variables are located to the left of `Y`

For explanation, refer to my comments and the original papers on linear programming (links above)

Canonical form is stated as follows:

$$
\begin{align}
Minimize: c^Tx\\
s.t. Ax=b\\
x\ge0
\end{align}
$$

Here is an example of unbounded problem:

Consider the following LPP:

$$
\begin{align}
Maximize: x_1+2x_2\\
s.t. -3x_1+2x_2\le2\\
-x_1+2x_2\le4\\
x_1, x_2\ge0
\end{align}
$$

In [10]:
c = [1, 2]
A = [
     [-3, 2],
     [-1, 2],
    ]

b = [2, 4]

x_opt, obj = simplex_method(A, b, c, verbose=True, optim='max')

Simplex Tableau
---------+-------------------------+---------
         |  1.00  2.00  0.00  0.00 | 0.00
---------+-------------------------+---------
x[2]       -3.00  2.00  1.00  0.00 | 2.00
x[3]       -1.00  2.00  0.00  1.00 | 4.00
---------+-------------------------+---------
Simplex Tableau
---------+-------------------------+---------
         |  4.00  0.00 -1.00  0.00 |-2.00
---------+-------------------------+---------
x[1]       -1.50  1.00  0.50  0.00 | 1.00
x[3]        2.00  0.00 -1.00  1.00 | 2.00
---------+-------------------------+---------
Simplex Tableau
---------+-------------------------+---------
         |  0.00  0.00  1.00 -2.00 |-6.00
---------+-------------------------+---------
x[1]        0.00  1.00 -0.25  0.75 | 2.50
x[0]        1.00  0.00 -0.50  0.50 | 1.00
---------+-------------------------+---------


RuntimeError: ignored

One can specify the optimization problem via a parameter `task='max'` or by pre-multiplying the `c` vector by a factor of `-1` (the first way basically involves the same but inside the function itself). In other words, the following code will produce the same result as above:

In [5]:
c = [-1, -2]
A = [
     [-3, 2],
     [-1, 2],
    ]

b = [2, 4]

x_opt, obj = simplex_method(A, b, c, verbose=True)

Simplex Tableau
---------+-------------------------+---------
         |  1.00  2.00  0.00  0.00 | 0.00
---------+-------------------------+---------
x[2]       -3.00  2.00  1.00  0.00 | 2.00
x[3]       -1.00  2.00  0.00  1.00 | 4.00
---------+-------------------------+---------
Simplex Tableau
---------+-------------------------+---------
         |  4.00  0.00 -1.00  0.00 |-2.00
---------+-------------------------+---------
x[1]       -1.50  1.00  0.50  0.00 | 1.00
x[3]        2.00  0.00 -1.00  1.00 | 2.00
---------+-------------------------+---------
Simplex Tableau
---------+-------------------------+---------
         |  0.00  0.00  1.00 -2.00 |-6.00
---------+-------------------------+---------
x[1]        0.00  1.00 -0.25  0.75 | 2.50
x[0]        1.00  0.00 -0.50  0.50 | 1.00
---------+-------------------------+---------


RuntimeError: ignored

Another feature I have not implemented yet is setting the basic variables set manually before feeding the function. This would also let one use equality constraints specified manually (consider the following example with degenerate solution)

In [11]:
c = [-3, -2, -1, -5, 0, 0, 0]
A = [
     [7, 3, 4, 1, 1, 0, 0],
     [2, 1, 1, 5, 0, 0, 0],
     [1, 4, 5, 2, 0, 0, 1],
    ]

b = [7, 3, 8]

x_opt, obj = simplex_method(A, b, c, verbose=True)

Simplex Tableau
---------+-------------------------------------------------------------+---------
         |  3.00  2.00  1.00  5.00  0.00  0.00  0.00  0.00  0.00  0.00 | 0.00
---------+-------------------------------------------------------------+---------
x[7]        7.00  3.00  4.00  1.00  1.00  0.00  0.00  1.00  0.00  0.00 | 7.00
x[8]        2.00  1.00  1.00  5.00  0.00  0.00  0.00  0.00  1.00  0.00 | 3.00
x[9]        1.00  4.00  5.00  2.00  0.00  0.00  1.00  0.00  0.00  1.00 | 8.00
---------+-------------------------------------------------------------+---------
Simplex Tableau
---------+-------------------------------------------------------------+---------
         |  1.00  1.00  0.00  0.00  0.00  0.00  0.00  0.00 -1.00  0.00 |-3.00
---------+-------------------------------------------------------------+---------
x[7]        6.60  2.80  3.80  0.00  1.00  0.00  0.00  1.00 -0.20  0.00 | 6.40
x[3]        0.40  0.20  0.20  1.00  0.00  0.00  0.00  0.00  0.20  0.00 | 0.60
x[9]        

In [14]:
c = [-3, -2, -1, -5]
A = [
     [7, 3, 4, 1],
     [2, 1, 1, 5],
     [1, 4, 5, 2],
    ]

b = [7, 3, 8]

x_opt, obj = simplex_method(A, b, c, verbose=True, optim='max')

Simplex Tableau
---------+-------------------------------------------+---------
         | -3.00 -2.00 -1.00 -5.00  0.00  0.00  0.00 | 0.00
---------+-------------------------------------------+---------
x[4]        7.00  3.00  4.00  1.00  1.00  0.00  0.00 | 7.00
x[5]        2.00  1.00  1.00  5.00  0.00  1.00  0.00 | 3.00
x[6]        1.00  4.00  5.00  2.00  0.00  0.00  1.00 | 8.00
---------+-------------------------------------------+---------
Optimal solution:  0.000  0.000  0.000  0.000  7.000  3.000  8.000 
Objective value: 0.0


Another example, followed by visualization with `plotly`!

In [ ]:
c = [1, 1]
A = [
     [1, 2],
     [0, 1],
    ]

b = [4, 1]

x_opt, obj = simplex_method(A, b, c, verbose=True)

Simplex Tableau
---------+-------------------------+---------
         | -1.00 -1.00  0.00  0.00 | 0.00
---------+-------------------------+---------
x[2]        1.00  2.00  1.00  0.00 | 4.00
x[3]        0.00  1.00  0.00  1.00 | 1.00
---------+-------------------------+---------
Optimal solution:  0.000  0.000  4.000  1.000 
Objective value: 0.0


In [15]:
c = [1, 1]
A = [
     [1, 2],
     [0, 1],
    ]

b = [4, 1]

x_opt, obj = simplex_method(A, b, c, verbose=True, optim='max')

Simplex Tableau
---------+-------------------------+---------
         |  1.00  1.00  0.00  0.00 | 0.00
---------+-------------------------+---------
x[2]        1.00  2.00  1.00  0.00 | 4.00
x[3]        0.00  1.00  0.00  1.00 | 1.00
---------+-------------------------+---------
Simplex Tableau
---------+-------------------------+---------
         |  0.00 -1.00 -1.00  0.00 |-4.00
---------+-------------------------+---------
x[0]        1.00  2.00  1.00  0.00 | 4.00
x[3]        0.00  1.00  0.00  1.00 | 1.00
---------+-------------------------+---------
Optimal solution:  4.000  0.000  0.000  1.000 
Objective value: -4.0


# **Vizualization**

In [16]:
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [53]:
# state 2D LPP
# and plot its solution and constraints
# in the original space

c = [-3, -2]

A = [
     [7, 1],
     [2, 5],
     [1, 2],
    ]

b = [7, 3, 8]

_ = simplex_method(A, b, c, verbose=True)

Simplex Tableau
---------+-------------------------------+---------
         |  3.00  2.00  0.00  0.00  0.00 | 0.00
---------+-------------------------------+---------
x[2]        7.00  1.00  1.00  0.00  0.00 | 7.00
x[3]        2.00  5.00  0.00  1.00  0.00 | 3.00
x[4]        1.00  2.00  0.00  0.00  1.00 | 8.00
---------+-------------------------------+---------
Simplex Tableau
---------+-------------------------------+---------
         |  0.00  1.57 -0.43  0.00  0.00 |-3.00
---------+-------------------------------+---------
x[0]        1.00  0.14  0.14  0.00  0.00 | 1.00
x[3]        0.00  4.71 -0.29  1.00  0.00 | 1.00
x[4]        0.00  1.86 -0.14  0.00  1.00 | 7.00
---------+-------------------------------+---------
Simplex Tableau
---------+-------------------------------+---------
         |  0.00  0.00 -0.33 -0.33  0.00 |-3.33
---------+-------------------------------+---------
x[0]        1.00  0.00  0.15 -0.03  0.00 | 0.97
x[1]        0.00  1.00 -0.06  0.21  0.00 | 0.21
x[4]    

In [145]:
def make_helper_funcs(A, b, c):

  def make_linear_function(coefs):
    coefs = np.asarray(coefs)
    def linear(X):
      X = np.asarray(X)
      return coefs.T @ X
    return linear

  def make_constraint(coefs, bias=.0):
    coefs = np.asarray(coefs)
    def F(X):
      X = np.asarray(X)
      return (bias - coefs[1:].T @ X) / coefs[0]
    return F

  constraints = [make_constraint(a, bias=b) for a, b in zip(A, b)]
  target = make_linear_function(c)
  return target, constraints

In [146]:
def plot_2D_LPP(A, b, c, with_solution=True, problem='all', axis_range=None):
  # axis_range should be a dict of lists of size 2, containing ranges for
  # axes in the format consumable by plotly (see the default values below)
  A, b, c = map(np.asarray, (A, b, c))

  if problem not in ('max', 'min', 'all'): raise ValueError('Expected min, max or all')
  if axis_range is None: axis_range = {
      'x': [-1, 5],
      'y': [-1, 5],
      'z': [-5, 5],
  }

  target, constraints = make_helper_funcs(A, b, c)

  # greate grid for axis in the original 2D space
  X = np.linspace(-10, 20, 1000)
  Y = np.linspace(-10, 20, 1000)
  YY = np.atleast_2d(Y)

  # helper lambda for axis lines
  F_ = lambda X: np.zeros_like(X)

  # I have no idea of how to properly
  # create the input for target function
  # (remember, that one takes a single
  # vector argument)
  rng = default_rng(seed=42)

  Xs = rng.choice(X, 100)
  Ys = rng.choice(Y, 100)
  XY = np.c_[Xs, Ys].T

  # make some beauty
  constraint_line=dict(color='slategray', width=10)
  axis_line=dict(color='seagreen', width=15)
  max_solution=dict(color='crimson', size=10)
  min_solution=dict(color='navy', size=10)
  solution_line=dict(width=10, dash='longdash')

  constraint_lines = [go.Scatter3d(name=f'$X_{i}$', x=F(YY), y=Y, z=np.zeros_like(X), opacity=.3, mode='lines', line=constraint_line) for i, F in enumerate(constraints, start=3)]

  data=[
        go.Mesh3d(name='Objective', x=Xs, y=Ys, z=target(XY), opacity=.5, color='plum'),
        go.Scatter3d(name='$X_{1}$', x=X, y=F_(X), z=np.zeros_like(X), opacity=.9, mode='lines', line=axis_line),
        go.Scatter3d(name='$X_{2}$', x=F_(Y), y=Y, z=np.zeros_like(X), opacity=.9, mode='lines', line=axis_line),
  ] + constraint_lines

  if with_solution:
    # plot two dots in each case: the point on the plane
    # and corresponding projection onto our polyhedral set formed by constraints
    if problem in ('min', 'all'):
      sol, obj = simplex_method(A, b, c, optim='min')
      sol = sol[:2]
      sol = np.c_[sol, sol]
      data += [go.Scatter3d(name='MIN Solution', x=sol[0], y=sol[1], z=[.0, obj], mode='markers+lines', marker=min_solution, line=solution_line)]

    if problem in ('max', 'all'):
      sol, obj = simplex_method(A, b, c, optim='max')
      obj = -obj
      sol = sol[:2]
      sol = np.c_[sol, sol]
      data += [go.Scatter3d(name='MAX Solution', x=sol[0], y=sol[1], z=[.0, obj], mode='markers+lines', marker=max_solution, line=solution_line)]

  fig = go.Figure(data=data)

  title = 'Linear Programming Problem'

  if with_solution: title += ' and its Solution'

  fig.update_layout(
      title=title,
      scene = dict(
          xaxis = dict(nticks=10, range=axis_range['x'], title='X_1'),
          yaxis = dict(nticks=10, range=axis_range['y'], title='X_2'),
          zaxis = dict(nticks=10, range=axis_range['z'], title='Objective')
          ),
      width=900,
      height=900,
      )

  return fig

In [147]:
# this particular problem is unbounded

c = [2, 1]
A = [
     [-2, 3],
     [-2, 1],
    ]

b = [-4, -2]

plot_2D_LPP(A, b, c, with_solution=False)

In [148]:
# the inverse problem, however, is bounded
# note the open cone formed by constraints

c = [2, 1]
A = [
     [-2, 3],
     [-2, 1],
    ]

b = [-4, -2]

plot_2D_LPP(A, b, c, with_solution=True, problem='min')

In [140]:
c = [-3, -2]

A = [
     [7, 1],
     [2, 5],
     [1, 2],
    ]

b = [7, 3, 8]

plot_2D_LPP(A, b, c, with_solution=True, problem='all')

In [141]:
# another example with more constraints

c = [-3, 1]
A = [
     [7, 3],
     [-2, 1],
     [1, 4],
     [2, 7]
    ]

b = [7, 3, 8, 3]


plot_2D_LPP(A, b, c, with_solution=True, problem='all')

# **Notes**

Remarkably, during coding I managed to find some new Python tricks:
for example, how `numpy` works with references and most notably with `+=`.
Also, I reviewed that one can simultaneously assign several values to the masked `np.ndarray`, like it is done in one of the cells below

Also, there be no more need in writing `np.any()` or `np.all()`, as these are present as array methods along with `argrmax()`!

Using unary `~` operator is broadcasted on arrays and basically reverts the bits of underlying data (as in C/C++). On `x86_64` this leads to inverting the sign and subtracting `1` (see below)

Writing `x = x + b` and `x += b` is not completely equivalent as these invoke different operators: the behavior will vary if `x` is mutable (e.g. `list` or `np.ndarray`)

Once again I had to mess with dot products and matmul. **REMEMBER**: if you desire to obtain dot product, you can simply write `a.T @ b` (more natural and neat to me) as this is literally what a dot product represents!


In [ ]:
~8

-9

In [ ]:
a = np.linspace(5, 10, 6)
b = (a > 7)
b

array([False, False, False,  True,  True,  True])

In [ ]:
a = np.linspace(5, 10, 6)

print((a < 8).any())
print((a < 8).all())

True
False


In [ ]:
a = 5

def f(a):
  a = 9

print(a)
f(a)
print(a)

5
5


In [ ]:
a = np.arange(10, 20)

idx = np.arange(3, 6)

a[idx] = np.zeros_like(idx)
a

array([10, 11, 12,  0,  0,  0, 16, 17, 18, 19])

In [ ]:
a = np.random.randn(5)
print(a)

def foo():
  b = a
  b = b + 100
  return b

def foo_():
  b = a
  b += 100
  return b

print(foo())
print(a)

print(foo_())
print(a)

[ 1.21371663  1.352519   -0.30597173  0.62163199 -1.52620767]
[101.21371663 101.352519    99.69402827 100.62163199  98.47379233]
[ 1.21371663  1.352519   -0.30597173  0.62163199 -1.52620767]
[101.21371663 101.352519    99.69402827 100.62163199  98.47379233]
[101.21371663 101.352519    99.69402827 100.62163199  98.47379233]


In [ ]:
a, b = np.random.randn(5), np.random.randn(5)
a.T @ b == np.dot(a, b)

True